In [5]:
import pandas as pd
from google.colab import files
uploaded=files.upload()

Saving help_desk_analysis.csv to help_desk_analysis.csv


In [6]:
df=pd.read_csv("help_desk_analysis.csv")
print(df.head())
print(df.info())
print(df.describe())

     ticket_id submission_time resolution_time  category assigned_analyst  \
0  TICKET-1000        00:36:16        00:46:41     Other       Toni Wiley   
1  TICKET-1001        00:27:59        00:18:06  Software      Tanya Jones   
2  TICKET-1002        00:32:02        00:50:37     Other      Sheila Ball   
3  TICKET-1003        00:47:34        00:38:19  Hardware     Cynthia Rich   
4  TICKET-1004        00:46:09        00:53:07  Software     Janice Burns   

                                         description priority  
0  Dream part subject until full. Brother century...     High  
1  Application X is crashing every time I open it...   Medium  
2  Black attack cold would page. Reality she war ...   Medium  
3  My laptop's keyboard isn't working properly, s...     High  
4  Application X is crashing every time I open it...     High  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---

In [12]:
#convert submisisiona nd resolution time to hours
def hms_to_hours(x):
  h, m, s = x.split(':')
  return int(h) + int(m)/60 + int(s)/3600
df['submission_hours'] = df['submission_time'].apply(hms_to_hours)
df['resolution_hours'] = df['resolution_time'].apply(hms_to_hours)
#calculate resolution duration in hours
df['resolution_duration_hours'] = df['resolution_hours'] - df['submission_hours']
df


,ticket_id,submission_time,resolution_time,category,assigned_analyst,description,priority,submission_hours,resolution_hours,resolution_duration_hours,submission_hour_int
0,TICKET-1000,00:36:16,00:46:41,Other,Toni Wiley,Dream part subject until full. Brother century...,High,0.604444,0.778056,0.173611,0
1,TICKET-1001,00:27:59,00:18:06,Software,Tanya Jones,Application X is crashing every time I open it...,Medium,0.466389,0.301667,-0.164722,0
2,TICKET-1002,00:32:02,00:50:37,Other,Sheila Ball,Black attack cold would page. Reality she war ...,Medium,0.533889,0.843611,0.309722,0
3,TICKET-1003,00:47:34,00:38:19,Hardware,Cynthia Rich,"My laptop's keyboard isn't working properly, s...",High,0.792778,0.638611,-0.154167,0
4,TICKET-1004,00:46:09,00:53:07,Software,Janice Burns,Application X is crashing every time I open it...,High,0.769167,0.885278,0.116111,0
...,...,...,...,...,...,...,...,...,...,...,...
145,TICKET-1145,00:49:39,00:50:14,Billing,Cynthia Rich,Suddenly effort she without without.,Critical,0.827500,0.837222,0.009722,0
146,TICKET-1146,00:03:22,00:54:19,Network,Amanda White,Can't connect to the Wi-Fi. It's showing 'No i...,High,0.056111,0.905278,0.849167,0
147,TICKET-1147,00:56:42,00:01:09,Security,Cynthia Rich,Look scene real owner agent whose specific. Al...,Low,0.945000,0.019167,-0.925833,0
148,TICKET-1148,00:22:14,00:50:12,Other,Adam Stone,You reflect radio. Especially speech wish inte...,Medium,0.370556,0.836667,0.466111,0


In [13]:
#Analyze: Ticket volume trends by hour
df['submission_hour_int'] = (df['submission_hours'] * 24).astype(int)
tickets_per_hour = df.groupby('submission_hour_int')['ticket_id'].count().reset_index(name='ticket_count')
print(tickets_per_hour)

    submission_hour_int  ticket_count
0                     0             5
1                     1             8
2                     2             5
3                     3             3
4                     4             4
5                     5             4
6                     6             5
7                     7             7
8                     8             8
9                     9             8
10                   10             4
11                   11            10
12                   12             5
13                   13             6
14                   14             3
15                   15             8
16                   16             8
17                   17             6
18                   18             5
19                   19            11
20                   20             8
21                   21             2
22                   22             9
23                   23             8


In [15]:
#Average resolution time by category & priority
df_valid = df[df['resolution_duration_hours'] >= 0]
avg_resolution_time = df_valid.groupby(['category', 'priority'])['resolution_duration_hours'].mean().reset_index()
avg_resolution_time['resolution_duration_hours'] = avg_resolution_time['resolution_duration_hours'].round(2)
print(avg_resolution_time)

    category  priority  resolution_duration_hours
0     Access       Low                       0.66
1     Access    Medium                       0.44
2    Billing  Critical                       0.01
3    Billing      High                       0.29
4    Billing       Low                       0.29
5    Billing    Medium                       0.45
6   Hardware      High                       0.16
7   Hardware       Low                       0.11
8   Hardware    Medium                       0.32
9    Network      High                       0.39
10   Network       Low                       0.42
11   Network    Medium                       0.35
12     Other      High                       0.14
13     Other       Low                       0.75
14     Other    Medium                       0.30
15  Security       Low                       0.33
16  Software  Critical                       0.07
17  Software      High                       0.09
18  Software       Low                       0.26


In [16]:
#Analyst performance
analyst_perf = df_valid.groupby('assigned_analyst')['resolution_duration_hours'].mean().reset_index()
analyst_perf['resolution_duration_hours'] = analyst_perf['resolution_duration_hours'].round(2)
analyst_perf = analyst_perf.sort_values(by='resolution_duration_hours')
print(analyst_perf)

  assigned_analyst  resolution_duration_hours
6   Natasha Harris                       0.17
7      Sheila Ball                       0.19
2      Autumn Ryan                       0.24
4     Janice Burns                       0.29
9       Toni Wiley                       0.30
8      Tanya Jones                       0.30
3     Cynthia Rich                       0.31
0       Adam Stone                       0.33
5      Marvin West                       0.40
1     Amanda White                       0.53


In [17]:
#Top keywords from descriptions
from collections import Counter
import re
# df_valid exists
# Combine all descriptions into a single string
all_descriptions = ' '.join(df_valid['description'].dropna()).lower()
# Remove punctuation
all_descriptions = re.sub(r'[^a-zA-Z0-9\s]', '', all_descriptions)
# Split into words
words = all_descriptions.split()
# Count occurrences
word_counts = Counter(words)
# Get top 20 keywords
top_keywords = word_counts.most_common(20)
# Convert to DataFrame for display/export
top_keywords_df = pd.DataFrame(top_keywords, columns=['Keyword', 'Count'])
print(top_keywords_df)

        Keyword  Count
0   application     25
1             x     25
2            is     25
3      crashing     25
4         every     25
5          time     25
6             i     25
7          open     25
8            it     25
9            my     22
10         cant     19
11           to     14
12          the     14
13           no     14
14      connect     13
15         wifi     13
16          its     13
17      showing     13
18     internet     13
19       access     13


In [23]:
#creating excel file to import these
!pip install xlsxwriter
with pd.ExcelWriter("help_desk_analysis.xlsx", engine='xlsxwriter') as writer:
  # 1️⃣ Summary Dashboard
    summary_data = {
        'Metric': ['Total Tickets', 'Average Resolution Time (hrs)'],
        'Value': [len(df_valid), round(df_valid['resolution_duration_hours'].mean(),2)]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary Dashboard', index=False)
    # 2️⃣ Data & Calculations
    df_valid.to_excel(writer, sheet_name='Data & Calculations', index=False)
    # 3️⃣ Detailed Analysis
    # We'll write each table at different rows to avoid overlap
    tickets_per_hour.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=0)
    avg_resolution_time.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=20)
    analyst_perf.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=40)
    top_keywords_df.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=60)
    # 4️⃣ Recommendations
    recommendations = [
        "Automate common 'Access' issues to reduce ticket volume.",
        "Prioritize recurring software issues identified in top keywords.",
        "Balance workload among analysts based on resolution performance.",
        "Provide training or support for analysts with higher resolution times.",
        "Adjust staffing during peak ticket hours based on volume trends."
    ]
    pd.DataFrame({'Recommendations': recommendations}).to_excel(writer, sheet_name='Recommendations', index=False)
    from google.colab import files
files.download("help_desk_analysis.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")

# 1️⃣ Ticket Volume Trends by Hour
plt.figure(figsize=(10,5))
colors = ['red' if count == tickets_per_hour['ticket_count'].max() else 'skyblue'
          for count in tickets_per_hour['ticket_count']]
plt.bar(tickets_per_hour['submission_hour_int'], tickets_per_hour['ticket_count'], color=colors)
plt.xlabel('Submission Hour (0-23)')
plt.ylabel('Number of Tickets')
plt.title('Ticket Volume Trends by Hour')
plt.xticks(range(0,24))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig("ticket_volume_chart.png", bbox_inches='tight')
plt.close()

# 2️⃣ Average Resolution Time by Category & Priority
plt.figure(figsize=(10,6))
sns.barplot(
    data=avg_resolution_time,
    x='category',
    y='resolution_duration_hours',
    hue='priority',        # hue is used here, so palette is fine
    palette='Set2'
)
plt.title('Average Resolution Time by Category & Priority')
plt.ylabel('Resolution Time (hours)')
plt.xlabel('Category')
plt.legend(title='Priority')
plt.savefig("resolution_time_chart.png", bbox_inches='tight')
plt.close()

# 3️⃣ Analyst Performance
plt.figure(figsize=(10,6))
sns.barplot(
    data=analyst_perf.sort_values('resolution_duration_hours'),
    x='assigned_analyst',
    y='resolution_duration_hours',
    hue=None,              # explicitly set hue to None to avoid warning
    palette='viridis'
)
plt.title('Analyst Performance: Average Resolution Time')
plt.ylabel('Resolution Time (hours)')
plt.xlabel('Analyst')
plt.xticks(rotation=45)
plt.savefig("analyst_perf_chart.png", bbox_inches='tight')
plt.close()

# 4️⃣ Top Keywords in Descriptions
plt.figure(figsize=(12,6))
sns.barplot(
    data=top_keywords_df.head(15),
    x='Keyword',
    y='Count',
    hue=None,              # explicitly set hue to None
    palette='magma'
)
plt.title('Top Keywords in Ticket Descriptions')
plt.ylabel('Frequency')
plt.xlabel('Keyword')
plt.xticks(rotation=45)
plt.savefig("top_keywords_chart.png", bbox_inches='tight')
plt.close()


/tmp/ipython-input-3664925354.py:38: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/tmp/ipython-input-3664925354.py:54: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


In [27]:
import pandas as pd

# Create Excel writer with xlsxwriter
with pd.ExcelWriter("help_desk_dashboard.xlsx", engine='xlsxwriter') as writer:
    # 1️⃣ Data & Calculations sheet
    df_valid.to_excel(writer, sheet_name='Data & Calculations', index=False)

    # 2️⃣ Summary Dashboard
    summary_df = pd.DataFrame({
        'Metric': ['Total Tickets', 'Average Resolution Time (hrs)', 'Peak Ticket Hour', 'Category with Highest Avg Resolution'],
        'Value': [
            len(df_valid),
            round(df_valid['resolution_duration_hours'].mean(), 2),
            tickets_per_hour.loc[tickets_per_hour['ticket_count'].idxmax(), 'submission_hour_int'],
            avg_resolution_time.groupby('category')['resolution_duration_hours'].mean().idxmax()
        ]
    })
    summary_df.to_excel(writer, sheet_name='Summary Dashboard', index=False, startrow=0, startcol=0)

    # Access workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets['Summary Dashboard']

    # 3️⃣ Insert charts images
    worksheet.insert_image('D2', 'ticket_volume_chart.png')
    worksheet.insert_image('D20', 'resolution_time_chart.png')
    worksheet.insert_image('D38', 'analyst_perf_chart.png')
    worksheet.insert_image('D56', 'top_keywords_chart.png')

    # 4️⃣ Detailed Analysis sheet
    tickets_per_hour.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=0)
    avg_resolution_time.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=20)
    analyst_perf.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=40)
    top_keywords_df.to_excel(writer, sheet_name='Detailed Analysis', index=False, startrow=60)

    # 5️⃣ Recommendations sheet
    recommendations = [
        "Automate common 'Access' issues to reduce ticket volume.",
        "Prioritize recurring software issues identified in top keywords.",
        "Balance workload among analysts based on resolution performance.",
        "Provide training or support for analysts with higher resolution times.",
        "Adjust staffing during peak ticket hours based on volume trends."
    ]
    pd.DataFrame({'Recommendations': recommendations}).to_excel(writer, sheet_name='Recommendations', index=False)
from google.colab import files
files.download("help_desk_dashboard.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>